In [1]:
param_dir = 'data_20240206'
param_make = 'ABARTH'

In [2]:
import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

In [3]:
input_ = pd.read_excel('./' + param_dir + '/' + param_make + '/6.type.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
input_['Type'].tolist()

总数量：2


['Coolant / Antifreeze', 'Brake Pad']

In [4]:
crawler_status = 'error'

print(crawler_status)

error


In [5]:
import requests
import json

In [6]:
import sys
sys.path.append('../../../00.module')
import UA
import Proxy

i = 0

jsn = '{"jsn":{"tab":"catalog","make":"' + input_.loc[i, 'Make'] + '","year":"' + input_.loc[i, 'Year'] + '","model":"' + input_.loc[i, 'Model'] + '","engine":"' + input_.loc[i, 'Engine'] + '","carcode":"' + input_.loc[i, 'Car_Code'] + '","groupname":"' + input_.loc[i, 'Category'] + '","parttype":"' + input_.loc[i, 'Type_Code'] + '","nodetype":"parttype"}}'

payload = {'func': 'navnode_fetch',
           'payload': jsn,
           'api_json_request': '1'}

resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                     data=payload,
                                         proxies=Proxy.get_Proxy_Requests()).text

resp = json.loads(resp)

resp

{'html_fill_sections': {'navchildren[]': '<div class="ranavnode" id="nav[__GIP__2__]"><input type="hidden" autocomplete="off" id="jsn[__GIP__2__]" value="{&quot;groupindex&quot;:&quot;__GIP__2__&quot;,&quot;tab&quot;:&quot;catalog&quot;,&quot;nodetype&quot;:&quot;no_parts_found&quot;,&quot;not_expandable&quot;:true}" /><div id="nav_o[__GIP__2__]" class="ranavouter"><div class="inner"><table class="tbl"  cellpadding="0" cellspacing="0"><tr><td class="niconspace nlasticon" id="navicon[__GIP__2__]"></td><td class="nlabel" ><span class="navlabellink nvoffset nnormal">No Parts Found</span></td></tr></table></div></div><div class="nchildren ra-hide" id="navchildren[__GIP__2__]" ></div></div>'},
 'jsn': {'tab': 'catalog',
  'make': 'ABARTH',
  'year': '1969',
  'model': '1000',
  'engine': '982cc L4',
  'carcode': '1438885',
  'groupname': 'Cooling System',
  'parttype': '4008823823',
  'nodetype': 'parttype'},
 'collected_javascript': '/**UQ:56322749**/\nwindow.top.parent.window["_nck"] = "8

In [7]:
from bs4 import BeautifulSoup

In [8]:
html = resp['html_fill_sections']['navchildren[]']

soup = BeautifulSoup(html, 'lxml')

with open('./part_number.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="ranavnode" id="nav[__GIP__2__]">
   <input autocomplete="off" id="jsn[__GIP__2__]" type="hidden" value='{"groupindex":"__GIP__2__","tab":"catalog","nodetype":"no_parts_found","not_expandable":true}'/>
   <div class="ranavouter" id="nav_o[__GIP__2__]">
    <div class="inner">
     <table cellpadding="0" cellspacing="0" class="tbl">
      <tr>
       <td class="niconspace nlasticon" id="navicon[__GIP__2__]">
       </td>
       <td class="nlabel">
        <span class="navlabellink nvoffset nnormal">
         No Parts Found
        </span>
       </td>
      </tr>
     </table>
    </div>
   </div>
   <div class="nchildren ra-hide" id="navchildren[__GIP__2__]">
   </div>
  </div>
 </body>
</html>



In [9]:
from lxml import etree

In [10]:
html = etree.HTML(str(soup))

list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]/tr/td/input[contains(@id, "listing_data_supplemental")]/@value')

list_part

[]

In [11]:
list_tag = [json.loads(part)['sortgrouptext'].strip() for part in list_part]

list_tag

[]

In [12]:
list_manufacturer = [json.loads(part)['catalogname'].strip() for part in list_part]

list_manufacturer

[]

In [13]:
list_part_number = [json.loads(part)['partnumber'].strip() for part in list_part]

list_part_number

[]

In [14]:
crawler_status = 'ok'

print(crawler_status)

ok


In [15]:
if crawler_status == 'ok':
    df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                            'No.': int(input_.loc[i, 'No.']),
                            'Make': input_.loc[i, 'Make'],
                            'Year': input_.loc[i, 'Year'],
                            'Model': input_.loc[i, 'Model'],
                            'Engine': input_.loc[i, 'Engine'],
                            'Car_Code': input_.loc[i, 'Car_Code'],
                            'Category': input_.loc[i, 'Category'],
                            'Subcategory': '',
                            'Type': input_.loc[i, 'Type'],
                            'Type_Code': input_.loc[i, 'Type_Code'],
                            'Tab_Url': input_.loc[i, 'Url'],
                            'Part_Index': [(index+1) for index in range(len(list_part))],
                            'Tag': list_tag,
                            'Manufacturer': list_manufacturer,
                            'Part_Number': list_part_number,
                            'Vehcile': '',
                            'OE': '',
                            'Note_1': '',
                            'Note_2': '',
                            'Pic': '',
                            'Url': '',
                            'Info': '',
                            'Src': '',
                            'Part_Type': '',
                            'Part_Key': '',
                            'JOIN_MPN': [manufacturer+';'+part_number for manufacturer, part_number in zip(list_manufacturer, list_part_number)]})
    
    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
else:
    df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                             'No.': int(input_.loc[i, 'No.']),
                             'Make': input_.loc[i, 'Make'],
                             'Year': input_.loc[i, 'Year'],
                             'Model': input_.loc[i, 'Model'],
                             'Engine': input_.loc[i, 'Engine'],
                             'Car_Code': input_.loc[i, 'Car_Code'],
                             'Category': input_.loc[i, 'Category'],
                             'Type': input_.loc[i, 'Type'],
                             'Type_Code': input_.loc[i, 'Type_Code'],
                             'Url': input_.loc[i, 'Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
    
print(len(output_correct), len(output_error))
output_correct

0 0


,Tab_Label,No.,Make,Year,Model,Engine,Car_Code,Category,Subcategory,Type,...,OE,Note_1,Note_2,Pic,Url,Info,Src,Part_Type,Part_Key,JOIN_MPN
